# 가고싶은곳 지도에 표시해보기

In [1]:
import requests
from urllib.parse import quote
import pandas as pd
import folium

In [2]:
with open('roadapikey.txt')as file:
    road_key = file.read()

In [3]:
quote('인천국제공항') 

'%EC%9D%B8%EC%B2%9C%EA%B5%AD%EC%A0%9C%EA%B3%B5%ED%95%AD'

In [4]:
road_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do'
params1 = f'confmKey={road_key}&currentPage=1&countPerPage=10'
params2 = f'keyword={quote("인천국제공항")}&resultType=json'
url = f"{road_url}?{params1}&{params2}"
url

'https://www.juso.go.kr/addrlink/addrLinkApi.do?confmKey=U01TX0FVVEgyMDIxMTIwMzEzNTIxODExMTk4ODM=&currentPage=1&countPerPage=10&keyword=%EC%9D%B8%EC%B2%9C%EA%B5%AD%EC%A0%9C%EA%B3%B5%ED%95%AD&resultType=json'

In [5]:
result = requests.get(url).json()
result

{'results': {'common': {'errorMessage': '정상',
   'countPerPage': '10',
   'totalCount': '10',
   'errorCode': '0',
   'currentPage': '1'},
  'juso': [{'detBdNmList': '',
    'engAddr': '24, Incheongukjegonghang Expressway, Seo-gu, Incheon',
    'rn': '인천국제공항고속도로',
    'emdNm': '검암동',
    'zipNo': '22690',
    'roadAddrPart2': '(검암동)',
    'emdNo': '01',
    'sggNm': '서구',
    'jibunAddr': '인천광역시 서구 검암동 436-15',
    'siNm': '인천광역시',
    'roadAddrPart1': '인천광역시 서구 인천국제공항고속도로 24',
    'bdNm': '',
    'admCd': '2826010300',
    'udrtYn': '0',
    'lnbrMnnm': '436',
    'roadAddr': '인천광역시 서구 인천국제공항고속도로 24(검암동)',
    'lnbrSlno': '15',
    'buldMnnm': '24',
    'bdKdcd': '0',
    'liNm': '',
    'rnMgtSn': '282601000029',
    'mtYn': '0',
    'bdMgtSn': '2826010300201130013000001',
    'buldSlno': '0'},
   {'detBdNmList': '',
    'engAddr': '271, Gonghang-ro, Jung-gu, Incheon',
    'rn': '공항로',
    'emdNm': '운서동',
    'zipNo': '22382',
    'roadAddrPart2': '(운서동)',
    'emdNo': '01',
    'sgg

In [6]:
places = '인천국제공항 한강공원 신선대 강원랜드 평창휘닉스 전주한옥마을 예술의전당 롯데월드 호미곶 휘닉스아일랜드'.split()
places

['인천국제공항',
 '한강공원',
 '신선대',
 '강원랜드',
 '평창휘닉스',
 '전주한옥마을',
 '예술의전당',
 '롯데월드',
 '호미곶',
 '휘닉스아일랜드']

In [7]:
addr_list = []
for place in places:
    road_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do'
    params1 = f'confmKey={road_key}&currentPage=1&countPerPage=10'
    params2 = f'keyword={quote("인천국제공항")}&resultType=json'
    url = f"{road_url}?{params1}&keyword={quote(place)}&resultType=json"
    result = requests.get(url).json()
    addr = result['results']['juso'][0]['roadAddr']
    addr_list.append(addr)

In [8]:
import pandas as pd

df = pd.DataFrame({'명소':places, '도로명주소':addr_list})

In [9]:
with open('kakaoapikey.txt') as file:
    kakao_key = file.read()

In [10]:
local_url = "https://dapi.kakao.com/v2/local/search/address.json"
addr = '인천광역시 서구 인천국제공항고속도로 24(검암동)'
url = f'{local_url}?query={quote(addr)}'
header = {"Authorization":f'KakaoAK {kakao_key}'}
result = requests.get(url, headers=header).json()
lat = float(result['documents'][0]['y'])
lng = float(result['documents'][0]['x'])

In [11]:
lng_list, lat_list = [], []
for i in df.index:
    url = f'{local_url}?query={quote(df.도로명주소[i])}'
    result = requests.get(url, headers=header).json()
    lat = float(result['documents'][0]['y'])
    lng = float(result['documents'][0]['x'])
    lng_list.append(lng)
    lat_list.append(lat)

df['위도'] = lat_list
df['경도'] = lng_list
df

,명소,도로명주소,위도,경도
0,인천국제공항,인천광역시 서구 인천국제공항고속도로 24(검암동),37.568995,126.662123
1,한강공원,서울특별시 강동구 선사로 83-106(암사동),37.549966,127.122249
2,신선대,부산광역시 남구 북항로 97(감만동),35.108351,129.083613
3,강원랜드,"서울특별시 서초구 효령로8길 32(방배동, 강원랜드)",37.476692,126.988944
4,평창휘닉스,강원도 평창군 봉평면 태기로 140,37.583405,128.325495
5,전주한옥마을,전라북도 전주시 완산구 한지길 89(풍남동3가),35.816363,127.154015
6,예술의전당,서울특별시 서초구 남부순환로 2406(서초동),37.480240,127.014215
7,롯데월드,서울특별시 송파구 올림픽로 300(신천동),37.513752,127.104447
8,호미곶,경상북도 포항시 남구 호미곶면 호미곶길 155-22,36.078500,129.563751
9,휘닉스아일랜드,제주특별자치도 서귀포시 성산읍 섭지코지로 107,33.426042,126.926443


In [22]:
map = folium.Map(location=[df.위도.mean(), df.경도.mean()], zoom_start=6)
for i in df.index:
    folium.Marker(
    location=[df.위도[i], df.경도[i]],
    tooltip= df.명소[i],
    popup=folium.Popup(df.도로명주소[i], max_width=200),
    icon=folium.Icon(color='purple' , icon = 'star')
).add_to(map)

title_html = '<h3 align="center" style="font-size:20px">연습용 명소 표시하기</h3>'   
map.get_root().html.add_child(folium.Element(title_html))
map